<a href="https://colab.research.google.com/github/rb58853/ML-RSI-Images/blob/main/code/consult_model/position_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [102]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Input, Dense, Reshape, Flatten,LSTM
from keras.models import Model,Sequential

import numpy as np

class MyModel():
  def __init__(self, data_path, max_input_len = 50, max_output_len = 50):
    self.max_input_len = max_input_len
    self.max_output_len = max_output_len

    self.data = self.get_data(data_path)
    self.model = self.get_model()

  def get_data(self,data_path):
    return {
    'input':[
        'a dog to the left of a cat',
        'a cat to the left of a lion',
        'a bed to the left of a table',
        'a sit to the left of a mantis',
        'a object to the left of another object',
        'a rock to the left of the water',
        'a sea to the left of a shark'
        ],
    'output':[
        ['a cat', 'a dog', '','',''],
        ['a lion', 'a cat', '','',''],
        ['a table', 'a bed', '','',''],
        ['a mantis', 'a sit', '','',''],
        ['another object', 'a object', '','',''],
        ['the water', 'a rock', '','',''],
        ['a shark', 'a sea', '','',''],
        ],
    'types':
     [
        [1,5,1,1,10,1,1,5],
        [1,5,1,1,10,1,1,5],
        [1,5,1,1,10,1,1,5],
        [1,5,1,1,10,1,1,5],
        [1,5,1,1,10,1,5,5],
        [1,5,1,1,10,1,1,5],
        [1,5,1,1,10,1,1,5],
        ]
      }

  def get_train(self):
    # Crea un objeto Tokenizer
    tokenizer = Tokenizer()
    # Ajusta el tokenizer a tus datos
    types =  self.data['types']
    output = self.data ['output']
    input =  self.data ['input']

    tokenizer.fit_on_texts(input+[':'])
    vocab_size = len(tokenizer.word_index) + 1
    sequences_input = tokenizer.texts_to_sequences(input)

    sequences_output = []
    for item in output:
      texts = [text for text in item]
      sequences_output.append(tokenizer.texts_to_sequences(texts))

    train_x =  [[date_input, date_type] for date_input, date_type in zip(sequences_input, types)]
    # train_x = np.array(train_x)
    train_y = [item for item in sequences_output]

    for case_ in train_x:
      for item in case_:
        while len(item) < self.max_input_len:
          item.append(-1)

    for case_ in train_y:
      for item in case_:
        while len(item) < self.max_output_len:
          item.append(-1)

    return  {'train_x':train_x, 'train_y':train_y}

  def get_model(self):
    total_len_output = self.max_output_len * 5

    model = Sequential()
    model.add(Input(shape=(2, self.max_input_len)))
    model.add(LSTM(256, activation='relu', return_sequences=True, input_shape=(100, self.max_input_len)))
    model.add(LSTM(256, activation='relu', return_sequences=True, input_shape=(100, self.max_input_len)))
    model.add(LSTM(256, activation='relu', return_sequences=True, input_shape=(100, self.max_input_len)))
    model.add(LSTM(256, activation='relu'))
    # model.add(Dense(128, activation='relu'))
    # model.add(Dense(64, activation='relu'))
    model.add(Flatten(input_shape=(0,64)))
    model.add(Dense(total_len_output, activation='relu'))
    model.add(Reshape((5, self.max_output_len)))
    return model

  def fit(self,epochs, batch_size=32):
    self.model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    train = self.get_train()
    train_x = train['train_x']
    train_y = train['train_y']
    self.model.fit(train_x, train_y, epochs=epochs, batch_size=32)
    # Convierte tus datos en secuencias de números


In [103]:
model = MyModel("data_path")

model.fit(100)

Epoch 1/100
1/1 [==============================] - 6s 6s/step - loss: -405.1151 - accuracy: 0.0000e+00
Epoch 2/100
1/1 [==============================] - 0s 77ms/step - loss: -603.7340 - accuracy: 0.0571
Epoch 3/100
1/1 [==============================] - 0s 81ms/step - loss: -645.1660 - accuracy: 0.2000
Epoch 4/100
1/1 [==============================] - 0s 79ms/step - loss: -668.1833 - accuracy: 0.3429
Epoch 5/100
1/1 [==============================] - 0s 70ms/step - loss: -686.3841 - accuracy: 0.5714
Epoch 6/100
1/1 [==============================] - 0s 71ms/step - loss: -693.0478 - accuracy: 0.5714
Epoch 7/100
1/1 [==============================] - 0s 75ms/step - loss: -696.8556 - accuracy: 0.5714
Epoch 8/100
1/1 [==============================] - 0s 76ms/step - loss: -696.1592 - accuracy: 0.5714
Epoch 9/100
1/1 [==============================] - 0s 78ms/step - loss: -698.6837 - accuracy: 0.5714
Epoch 10/100
1/1 [==============================] - 0s 83ms/step - loss: -707.4380 - accu